# Named Entity Recognition
- using conditional random fields

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [0]:
# For data processing and maths

import pandas as pd
import numpy as np
#For Visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [6]:
! pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 9.4MB/s 


In [9]:
! pip install eli5

     |████████████████████████████████| 112kB 6.2MB/s 


In [10]:
# for model

from sklearn_crfsuite import CRF
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
import eli5

Using TensorFlow backend.


In [11]:
! pip install version_information

In [12]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,numpy,sklearn, eli5, seaborn, matplotlib

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
numpy 1.17.4
sklearn 0.21.3
eli5 0.10.1
seaborn 0.9.0
matplotlib 3.1.1
Tue Nov 26 19:41:49 2019 UTC

# 2)- Getting Data

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [0]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [14]:
data.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O

There are missing values. We shall fill wit ffill. Also it makes sense as sentence 1 contains all words and then comes sentence2.

In [0]:
data = data.fillna(method="ffill")

In [16]:
data.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1  Sentence: 1             of   IN   O
2  Sentence: 1  demonstrators  NNS   O
3  Sentence: 1           have  VBP   O
4  Sentence: 1        marched  VBN   O

In [17]:
data.tail()

Sentence #       Word  POS Tag
1048570  Sentence: 47959       they  PRP   O
1048571  Sentence: 47959  responded  VBD   O
1048572  Sentence: 47959         to   TO   O
1048573  Sentence: 47959        the   DT   O
1048574  Sentence: 47959     attack   NN   O

So, in total we have 47959 sentences in our data. Let's see how many words we have

In [18]:
len(data["Word"])

1048575

Total words are 1048575. But , there might be many repeating words. Let's find unique words or vocab.

In [19]:
len(data["Word"].unique())

35178

In [0]:
# making list of unique "word" feature
words = list(set(data["Word"].values))

In [21]:
words[:5]

['Tommy', 'acclaim', 'al-Dabbagh', 'lived', 'ultra-orthodox']

In [22]:
n_words = len(words)
n_words

35178

We have 47959 sentences containing 35178 unique words

In [23]:
# let's see how many tag we have in our data or what kind of tags we have

data.Tag.value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [24]:
len(data.Tag.value_counts())

17

In [25]:
# how about POS

len(data.POS.value_counts())

42

#3)- Retrieving sentence

- SentenceGetter function

In [0]:
d=data

In [0]:
d=d.groupby(["Sentence #","Tag"]).size()

In [20]:
d.head()

Sentence #    Tag  
Sentence: 1   B-geo     2
              B-gpe     1
              O        21
Sentence: 10  B-gpe     1
              B-org     1
dtype: int64

In [0]:
d_tag=d.unstack('Tag')

In [22]:
d_tag

Tag              B-art  B-eve  B-geo  B-gpe  ...  I-org  I-per  I-tim     O
Sentence #                                   ...                           
Sentence: 1        NaN    NaN    2.0    1.0  ...    NaN    NaN    NaN  21.0
Sentence: 10       NaN    NaN    NaN    1.0  ...    NaN    NaN    NaN  22.0
Sentence: 100      NaN    NaN    2.0    NaN  ...    NaN    NaN    NaN  27.0
Sentence: 1000     NaN    NaN    NaN    NaN  ...    NaN    NaN    NaN  11.0
Sentence: 10000    NaN    NaN    3.0    2.0  ...    NaN    1.0    NaN  27.0
...                ...    ...    ...    ...  ...    ...    ...    ...   ...
Sentence: 9995     NaN    NaN    NaN    NaN  ...    NaN    3.0    NaN  27.0
Sentence: 9996     NaN    NaN    NaN    1.0  ...    1.0    NaN    NaN  21.0
Sentence: 9997     NaN    NaN    1.0    NaN  ...    NaN    NaN    1.0  24.0
Sentence: 9998     NaN    NaN    NaN    NaN  ...    NaN    NaN    NaN  21.0
Sentence: 9999     NaN    NaN    NaN    NaN  ...    1.0    NaN    1.0  28.0

[47959 rows x 17 columns]

Thanks to blog by http://www.depends-on-the-definition.com/introduction-named-entity-recognition-python/

We can use a simpler solution for this 

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)

In [0]:
sent = getter.get_next()

In [29]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [0]:
# get all sentences in dataset
sentences = getter.sentences

# 4)- Feature preparation

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
# splitting data into X and y

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

# 5)- CRF model

In [0]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [37]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

       B-art       0.37      0.11      0.17       402
       B-eve       0.52      0.35      0.42       308
       B-geo       0.85      0.90      0.88     37644
       B-gpe       0.97      0.94      0.95     15870
       B-nat       0.66      0.37      0.47       201
       B-org       0.78      0.72      0.75     20143
       B-per       0.84      0.81      0.82     16990
       B-tim       0.93      0.88      0.90     20333
       I-art       0.11      0.03      0.04       297
       I-eve       0.34      0.21      0.26       253
       I-geo       0.82      0.79      0.80      7414
       I-gpe       0.92      0.55      0.69       198
       I-nat       0.61      0.27      0.38        51
       I-org       0.81      0.79      0.80     16784
       I-per       0.84      0.89      0.87     17251
       I-tim       0.83      0.76      0.80      6528
           O       0.99      0.99      0.99    887908

    accuracy              

# 6)- Feature importance 

given by predicted results from CRF model.

In [38]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [39]:
# using eli5
eli5.show_weights(crf, top=30)

# 7)- Improving results

applying regularization in CRF model

**CRF is showing results where it remembers a lot of words.we can tune the parameters, especially the regularization parameters of the CRF algorithm. The c1 and c2 parameter of the CRF algorithm are the regularization parameters**

In [0]:
crf = CRF(algorithm='lbfgs',
          c1=10, # changing value
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [42]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       402
       B-eve       0.80      0.27      0.40       308
       B-geo       0.82      0.90      0.86     37644
       B-gpe       0.95      0.92      0.94     15870
       B-nat       0.69      0.09      0.16       201
       B-org       0.78      0.67      0.72     20143
       B-per       0.80      0.76      0.78     16990
       B-tim       0.93      0.83      0.88     20333
       I-art       0.00      0.00      0.00       297
       I-eve       0.64      0.12      0.20       253
       I-geo       0.81      0.73      0.77      7414
       I-gpe       0.93      0.37      0.53       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.75      0.76      0.75     16784
       I-per       0.80      0.90      0.85     17251
       I-tim       0.84      0.67      0.74      6528
           O       0.99      0.99      0.99    887908

    accuracy              

In [43]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=10, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [44]:
eli5.show_weights(crf, top=30)